---
title: Numerical Analysis
math:
    '\abs': '\left\lvert #1 \right\rvert'
    '\orm': '\left\lvert #1 \right\rvert'
    '\Set': '\left\{ #1 \right\}'
    '\set': '\operatorname{set}'   
    '\mc': '\mathcal{#1}'
    '\M': '\boldsymbol{#1}'
    '\R': '\mathsf{#1}'
    '\RM': '\boldsymbol{\mathsf{#1}}'
    '\op': '\operatorname{#1}'
    '\E': '\op{E}'
    '\d': '\mathrm{\mathstrut d}'
    '\SFM': '\operatorname{SFM}'
    '\utag': '\stackrel{\text{(#1)}}{#2}'
    '\uref': '\text{(#1)}'
    '\minimal': '\operatorname{minimal}'
abstract: |
    An important engineering application of computer science is numerical analysis. This lecture introduces estimation using Monte Carlo simulations and basic linear algebra for solving multiple linear equations. Readers will learn to use NumPy arrays instead of lists to implement numerical analysis efficiently, incorporating the concepts of universal functions and broadcasting.
---

In [ ]:
from __init__ import install_dependencies

await install_dependencies()

In [ ]:
import math
import random

import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt

%matplotlib widget
%reload_ext divewidgets
%load_ext jupyter_ai
%ai update chatgpt dive:chat

## Monte Carlo simulation

**What is [Monte Carlo simulation](https://en.wikipedia.org/wiki/Monte_Carlo_method)?**

It is a statistical method named after a [Casino in Monaco](https://en.wikipedia.org/wiki/Monte_Carlo_Casino):

> I suggested an obvious name for the statistical method — a suggestion not unrelated to the fact that Stan had an uncle who would borrow money from relatives because he "just had to go to Monte
Carlo."
> 
> (N. Metropolis, "[The Beginning of the Monte Carlo Method](https://permalink.lanl.gov/object/tr?what=info:lanl-repo/lareport/LA-UR-88-9067).")

It would be nice to simulate the casino so Ulam's uncle did not need to borrow money to play in the casino.

Actually...,

- Monte Carlo is the code name of the secret project for creating the [hydrogen bomb](https://en.wikipedia.org/wiki/Monte_Carlo_method). 
- [Nicholas Metropolis and Stan Ulam](https://doi.org/10.2307/2280232) worked with [John von Neumann](https://en.wikipedia.org/wiki/John_von_Neumann) to program the first electronic computer ENIAC to simulate a computational model of a thermonuclear reaction.

As a simple illustration, we will compute the value of $\pi$:

In [ ]:
math.pi

Instead of using `pi` from `math`, can we compute $\pi$ directly? Consider a seemingly unrelated math problem:

::::{admonition} What is the chance a random point in a square lies in the inscribed circle?
:class: dropdown

Suppose the square has length $\ell$. The chance is

$$ P[\text{Random point in inscribed circle}] = \frac{\text{area of circle}}{\text{area of square}} = \frac{\pi (\ell /2)^2}{ (\ell)^2 } = \frac{\pi}4 $$

independent of the length $\ell$.

::::

::::{figure} images/RandomPointInSquare.gif
:name:
:alt: Random points in square.
:align: left

Uniformly random points in a square. Green (Red) points are inside (outside) the inscribed circle. ([source code](https://dive4dec.github.io/optlite/#code=from%20manim%20import%20*%0A%0A%23%20%25%25manim%20-ql%20-t%20--progress_bar%20None%20--disable_caching%20-v%20ERROR%20RandomPointInSquare%0Aclass%20RandomPointInSquare%28Scene%29%3A%0A%20%20%20%20def%20construct%28self%29%3A%0A%20%20%20%20%20%20%20%20%23%20Set%20the%20background%20color%20to%20white%0A%20%20%20%20%20%20%20%20self.camera.background_color%20%3D%20WHITE%0A%20%20%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20%23%20Define%20the%20side%20length%20of%20the%20square%0A%20%20%20%20%20%20%20%20side_length%20%3D%206%0A%20%20%20%20%20%20%20%20radius%20%3D%20side_length%20/%202%0A%0A%20%20%20%20%20%20%20%20%23%20Create%20the%20square%0A%20%20%20%20%20%20%20%20square%20%3D%20Square%28side_length%3Dside_length,%20color%3DBLUE%29%0A%20%20%20%20%20%20%20%20square_label%20%3D%20MathTex%28r%22%5Cell%22,%20color%3DBLUE_D%29.next_to%28square,%20RIGHT%29%0A%0A%20%20%20%20%20%20%20%20%23%20Create%20the%20inscribed%20circle%0A%20%20%20%20%20%20%20%20circle%20%3D%20Circle%28radius%3Dradius,%20color%3DRED%29.move_to%28square.get_center%28%29%29%0A%0A%20%20%20%20%20%20%20%20%23%20Add%20the%20square%20and%20circle%20to%20the%20scene%0A%20%20%20%20%20%20%20%20self.play%28Create%28square%29%29%0A%20%20%20%20%20%20%20%20self.play%28Create%28circle%29%29%0A%20%20%20%20%20%20%20%20self.play%28Write%28square_label%29%29%0A%0A%20%20%20%20%20%20%20%20%23%20Simulate%20random%20points%0A%20%20%20%20%20%20%20%20num_points%20%3D%20100%0A%20%20%20%20%20%20%20%20points_inside%20%3D%200%0A%20%20%20%20%20%20%20%20for%20_%20in%20range%28num_points%29%3A%0A%20%20%20%20%20%20%20%20%20%20%20%20%23%20Generate%20a%20random%20point%0A%20%20%20%20%20%20%20%20%20%20%20%20x%20%3D%20np.random.uniform%28-radius,%20radius%29%0A%20%20%20%20%20%20%20%20%20%20%20%20y%20%3D%20np.random.uniform%28-radius,%20radius%29%0A%20%20%20%20%20%20%20%20%20%20%20%20point%20%3D%20Dot%28point%3D%5Bx,%20y,%200%5D%29%0A%20%20%20%20%20%20%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20%20%20%20%20%23%20Check%20if%20the%20point%20is%20inside%20the%20circle%0A%20%20%20%20%20%20%20%20%20%20%20%20if%20x**2%20%2B%20y**2%20%3C%3D%20radius**2%3A%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20point.set_color%28GREEN%29%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20points_inside%20%2B%3D%201%0A%20%20%20%20%20%20%20%20%20%20%20%20else%3A%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20point.set_color%28RED%29%0A%0A%20%20%20%20%20%20%20%20%20%20%20%20self.add%28point%29%0A%20%20%20%20%20%20%20%20%20%20%20%20self.wait%280.1%29&mode=edit&origin=opt-frontend.js&rawInputLstJSON=%5B%5D))

::::

By relating the chance of a random event to the quantity we want to compute, we can compute the quantity by simulating the random event. This method is called the Monte Carlo simulation:

::::{exercise}
:label: ex:ml1

Complete the following function to return an approximation of $\pi$ as follows:
1. Simulate the random process of picking a point from a square repeatedly `n` times by  
  generating the $x$ and $y$ coordinates uniformly randomly from a unit interval $[0,1)$.
2. Compute the fraction of times the point is in the first quadrant of the inscribed circle, as shown in the figure below.
3. Return $4$ times the fraction as the approximation.

:::{seealso} Buffon's needle

A similar way is to approximate $\pi$ is the [Buffon's needle](https://en.wikipedia.org/wiki/Buffon%27s_needle_problem). See [such a program](https://www.khanacademy.org/computer-programming/pi-by-buffons-needle/6695500989890560) written in javascript.

:::

::::

In [ ]:
def approximate_pi(n):
    # YOUR CODE HERE
    raise NotImplementedError

In [ ]:
%%timeit -n 1 -r 1
print(f"Approximate: {approximate_pi(10**7)}\nGround truth: {math.pi}")

**How accurate is the approximation?**

The following uses a powerful library [NumPy](https://numpy.org/) to return a [$95\%$-confidence interval](http://onlinestatbook.com/2/estimation/mean.html#:~:text=To%20compute%20the%2095%25%20confidence,be%20between%20the%20cutoff%20points.).

In [ ]:
def np_approximate_pi(n):
    in_circle = (np.random.random((n, 2)) ** 2).sum(axis=-1) < 1
    mean = 4 * in_circle.mean()
    std = 4 * in_circle.std() / n ** 0.5
    return np.array([mean - 2 * std, mean + 2 * std])

In [ ]:
%%timeit -n 1 -r 1
interval = np_approximate_pi(10**7)
print(
    f"""95%-confidence interval: {interval}
Estimate: {interval.mean():.4f} ± {(interval[1]-interval[0])/2:.4f}
Ground truth: {math.pi}"""
)

::::{note} Is Numpy `array` faster than Python `list`? 
:class: dropdown

The computation done using NumPy is over $5$ times faster despite the additional computation of the standard deviation.

::::

We can create $n$ further to obtain a more accurate solution:

In [ ]:
interval = np_approximate_pi(10**8)
print(
    f"""95%-confidence interval: {interval}
Estimate: {interval.mean():.4f} ± {(interval[1]-interval[0])/2:.4f}
Ground truth: {math.pi}"""
)

::::{seealso}

There are faster methods to approximate $\pi$, such as the [BBP Formula](https://en.wikipedia.org/wiki/Bailey%E2%80%93Borwein%E2%80%93Plouffe_formula) and [Chudnovsky algorithm](https://en.wikipedia.org/wiki/Chudnovsky_algorithm), but the Monte-Carlo method is helpful in complicated situations.

::::

In [ ]:
%%ai chatgpt -f text
Write a python program `print_pi(n)` to print pi to `n` decimal places.
Avoid the precision limit of float.

In [ ]:
%%ai chatgpt -f text
Provide an example of an application where Monte Carlo simulation is the only feasible computational method and no superior alternatives are known.

In [ ]:
%%ai chatgpt -f text
Explain how the accuracy of the estimates by Monte Carlo simulation can be improved by importance sampling?

## Linear Algebra

**How to solve a linear equation?**

Given the following linear equation in variable $x$ with real-valued coefficient $a$ and $b$,

$$ a x = b,$$
what is the value of $x$ that satisfies the equation?

::::{exercise}
:label: ex:ml2

Complete the following function to return either the unique solution of $x$ or `None` if a unique solution does not exist.

::::

In [ ]:
def solve_linear_equation(a, b):
    # YOUR CODE HERE
    raise NotImplementedError


@widgets.interact(a=(0, 5, 1), b=(0, 5, 1))
def linear_equation_solver(a=2, b=3):
    print(
        f"""linear equation: {a}x = {b}
       solution: x = {solve_linear_equation(a,b)}"""
    )

**How to solve multiple linear equations?**

In the general case, we have a system of $m$ linear equations and $n$ variables:

$$ \begin{aligned}
a_{00} x_0 + a_{01} x_1 + \dots + a_{0(n-1)} x_{n-1} &= b_0\\
a_{10} x_0 + a_{11} x_1 + \dots + a_{1(n-1)} x_{n-1} &= b_1\\
 & \vdots\\
a_{(m-1)0} x_0 + a_{(m-1)1} x_1 + \dots + a_{(m-1)(n-1)} x_{n-1} &= b_{m-1}\\
\end{aligned}
$$
where
- $x_j$ for $j\in \{0,\dots,n-1\}$ are the variables, and
- $a_{ij}$ and $b_j$ for $i\in \{0,\dots,m-1\}$ and $j\in \{0,\dots,n-1\}$ are the coefficients.

A fundamental problem in linear algebra is to compute the unique solution to the system if it exists.

We will consider the simpler 2-by-2 system with 2 variables and 2 equations:

$$ \begin{aligned}
a_{00} x_0 + a_{01} x_1 &= b_0\\
a_{10} x_0 + a_{11} x_1 &= b_1.
\end{aligned}
$$

To get an idea of the solution, suppose 

$$a_{00}=a_{11}=1, a_{01} = a_{10} = 0.$$
The system of equations becomes

$$ \begin{aligned}
x_0 \hphantom{+ x_1} &= b_0\\
\hphantom{x_0 +}  x_1 &= b_1,
\end{aligned}
$$
which gives the solution directly.

What about $a_{00}=a_{11}=2$ instead?

$$ \begin{aligned}
2x_0 \hphantom{+ x_1} &= b_0\\
\hphantom{2x_0 +}  2x_1 &= b_1,
\end{aligned}$$

To obtain the solution, we divide both equations by 2:

$$ \begin{aligned}
x_0 \hphantom{+ x_1} &= \frac{b_0}2\\
\hphantom{x_0 +}  x_1 &= \frac{b_1}2.
\end{aligned}
$$

What if $a_{01}=2$ instead?

$$ \begin{aligned}
2x_0 + 2x_1 &= b_0\\
\hphantom{2x_0 +}  2x_1 &= b_1\\
\end{aligned}
$$

The second equation gives the solution of $x_1$, and we can use the solution in the first equation to solve for $x_0$. More precisely:
- Subtract the second equation from the first one:

$$ \begin{aligned}
2x_0 \hphantom{+2x_1} &= b_0 - b_1\\
\hphantom{2x_0 +}  2x_1 &= b_1\\
\end{aligned}
$$
- Divide both equations by 2:

$$ \begin{aligned}
x_0 \hphantom{+ x_1} &= \frac{b_0 - b_1}2\\
\hphantom{x_0 +}  x_1 &= \frac{b_1}2\\
\end{aligned}
$$

The above operations are called *row operations* in linear algebra: Each row is an equation.

::::{important} Row operations

A system of linear equations can be solved by the linear operations of 
1. multiplying an equation by a constant, and
2. subtracting one equation from another.
::::

How to write a program to solve a general 2-by-2 system? We will use the NumPy library.

### Creating NumPy arrays

**How to store the coefficients?**

In linear algebra, a system of equations such as

$$ \begin{aligned}
a_{00} x_0 + a_{01} x_1 &= b_0\\
a_{10} x_0 + a_{11} x_1 &= b_1
\end{aligned}
$$
is written concisely in *matrix* form as $ \M{A} \M{x} = \M{b} $:

$$\overbrace{\begin{bmatrix}
a_{00} & a_{01}\\
a_{10} & a_{11}
\end{bmatrix}}^{\M{A}}
\overbrace{
\begin{bmatrix}
x_0\\
x_1
\end{bmatrix}}
^{\M{x}}
= \overbrace{\begin{bmatrix}
b_0\\
b_1
\end{bmatrix}}^{\M{b}},
$$
where
$ \M{A} \M{x}$ is the *matrix multiplication*

$$ \M{A} \M{x} = \begin{bmatrix}
a_{00} x_0 + a_{01} x_1\\
a_{10} x_0 + a_{11} x_1
\end{bmatrix}.
$$

::::{figure} images/LinearEquationToMatrixForm.gif
:name:
:alt: Matrix form
:align: left

Matrix form ([source code](https://dive4dec.github.io/optlite/#code=from%20manim%20import%20*%0A%0A%23%20%25%25manim%20--format%20gif%20-qm%20-t%20--progress_bar%20None%20--disable_caching%20-v%20ERROR%20LinearEquationToMatrixForm%0Aclass%20LinearEquationToMatrixForm%28Scene%29%3A%0A%20%20%20%20def%20construct%28self%29%3A%0A%20%20%20%20%20%20%20%20%23%20Set%20the%20background%20color%20to%20white%0A%20%20%20%20%20%20%20%20self.camera.background_color%20%3D%20WHITE%0A%20%20%20%20%20%20%20%20Text.set_default%28color%3DBLACK%29%0A%20%20%20%20%20%20%20%20MathTex.set_default%28color%3DBLACK%29%0A%0A%20%20%20%20%20%20%20%20%23%20Define%20the%20system%20of%20linear%20equations%0A%20%20%20%20%20%20%20%20equations%20%3D%20MathTex%28%0A%20%20%20%20%20%20%20%20%20%20%20%20r%22%7B%7Ba_%7B00%7D%7D%7D%7B%7Bx_0%7D%7D%20%2B%20%7B%7Ba_%7B01%7D%7D%7D%7B%7Bx_1%7D%7D%20%26%7B%7B%3D%7D%7D%20%7B%7Bb_0%7D%7D%20%5C%5C%22,%0A%20%20%20%20%20%20%20%20%20%20%20%20r%22%7B%7Ba_%7B10%7D%7D%7D%7B%7Bx_0%7D%7D%20%2B%20%7B%7Ba_%7B11%7D%7D%7D%7B%7Bx_1%7D%7D%20%26%7B%7B%3D%7D%7D%20%7B%7Bb_1%7D%7D%22,%0A%20%20%20%20%20%20%20%20%29.shift%28UP%20*%202%29%0A%0A%20%20%20%20%20%20%20%20%23%20Define%20the%20matrix%20form%0A%20%20%20%20%20%20%20%20a_%20%3D%20%5B%5BMathTex%28f%22%7B%7B%7B%7Ba_%7B%7B%7Bi%7D%7Bj%7D%7D%7D%7D%7D%7D%7D%22%29%20for%20j%20in%20range%282%29%5D%20for%20i%20in%20range%282%29%5D%0A%20%20%20%20%20%20%20%20x_%20%3D%20%5B%5BMathTex%28f%22%7B%7B%7B%7Bx_%7Bi%7D%7D%7D%7D%7D%22%29%5D%20for%20i%20in%20range%282%29%5D%0A%20%20%20%20%20%20%20%20b_%20%3D%20%5B%5BMathTex%28f%22%7B%7B%7B%7Bb_%7Bi%7D%7D%7D%7D%7D%22%29%5D%20for%20i%20in%20range%282%29%5D%0A%20%20%20%20%20%20%20%20eq%20%3D%20MathTex%28%22%7B%7B%3D%7D%7D%22%29%0A%20%20%20%20%20%20%20%20A%20%3D%20MobjectMatrix%28a_%29%0A%20%20%20%20%20%20%20%20x%20%3D%20MobjectMatrix%28x_%29%0A%20%20%20%20%20%20%20%20b%20%3D%20MobjectMatrix%28b_%29%0A%20%20%20%20%20%20%20%20matrix_form%20%3D%20VGroup%28A,%20x,%20eq,%20b%29.arrange_submobjects%28%29.shift%28DOWN%29%0A%0A%20%20%20%20%20%20%20%20%23%20Add%20the%20initial%20equations%20to%20the%20scene%0A%20%20%20%20%20%20%20%20self.add%28equations%29%0A%20%20%20%20%20%20%20%20self.wait%281%29%0A%0A%20%20%20%20%20%20%20%20%23%20Transform%20the%20equations%20into%20matrix%20form%0A%20%20%20%20%20%20%20%20self.play%28%0A%20%20%20%20%20%20%20%20%20%20%20%20TransformMatchingTex%28%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20equations,%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20Group%28%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20*%5Ba_%5Bi%5D%5Bj%5D%20for%20i%20in%20range%282%29%20for%20j%20in%20range%282%29%5D,%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20*%5Bx_%5Bi%5D%5B0%5D%20for%20i%20in%20range%282%29%5D,%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20eq,%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20*%5Bb_%5Bi%5D%5B0%5D%20for%20i%20in%20range%282%29%5D,%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%29,%0A%20%20%20%20%20%20%20%20%20%20%20%20%29%0A%20%20%20%20%20%20%20%20%29%0A%20%20%20%20%20%20%20%20self.add%28matrix_form%29%0A%20%20%20%20%20%20%20%20self.wait%282%29&mode=edit&origin=opt-frontend.js&rawInputLstJSON=%5B%5D))

::::

We say that $\M{A}$ is a [*matrix*](https://en.wikipedia.org/wiki/Matrix_(mathematics)) and its dimension/shape is $2$-by-$2$:
- The first dimension/axis has size $2$. We also say that the matrix has $2$ rows.
- The second dimension/axis has size $2$. We also say that the matrix has $2$ columns.
$\M{x}$ and $\M{b}$ are called column vectors, which are matrices with one column.

Consider the example

$$ \begin{aligned}
2x_0 + 2x_1 &= 1\\
\hphantom{2x_0 +}  2x_1 &= 1,
\end{aligned}$$

or in matrix form with

$$ \begin{aligned}
\M{A}&=\begin{bmatrix}
a_{00} & a_{01} \\
a_{10} & a_{11} 
\end{bmatrix} 
= \begin{bmatrix}
2 & 2 \\
0 & 2 
\end{bmatrix}\\
\M{b}&=\begin{bmatrix}
b_0\\
b_1
\end{bmatrix} = \begin{bmatrix}
1\\
1
\end{bmatrix}\end{aligned}$$

Instead of using `list` to store the matrix, we will use a NumPy array.

In [ ]:
A = np.array([[2.0, 2], [0, 2]])
b = np.array([1.0, 1])
A, b

Compared to `list`, NumPy array is often more efficient and has more attributes.

In [ ]:
array_attributes = set(attr for attr in dir(np.array([])) if attr[0] != "_")
list_attributes = set(attr for attr in dir(list) if attr[0] != "_")
print("\nCommon attributes:\n", *(array_attributes & list_attributes))
print("\nArray-specific attributes:\n", *(array_attributes - list_attributes))
print("\nList-specific attributes:\n", *(list_attributes - array_attributes))

The following attributes give the dimension/shape, number of dimensions, size, and data type.

In [ ]:
for array in A, b:
    print(
        f"""{array}
    shape: {array.shape}
    ndim: {array.ndim}
    size: {array.size}
    dtype: {array.dtype}
    """
    )

Note that the function `len` only returns the size of the first dimension:

In [ ]:
assert A.shape[0] == len(A)
len(A)

Unlike `list`, every NumPy array has a data type. For efficient computation/storage, numpy implements different data types with different storage sizes:

In [ ]:
np.byte, np.short, np.intc
np.ubyte, np.ushort, np.uintc, np.uint
np.half, np.single, np.double, np.longfloat
np.csingle, np.cdouble, np.clongdouble

E.g., `int64` is the 64-bit integer. Unlike `int`, `int64` has a range.

In [ ]:
%%optlite -h 400
import numpy as np
min_int64 = np.int64(-2**63)
max_int64 = np.int64(2**63-1)
np.int64(2 ** 63)  # overflow error

We can use the `astype` method to convert the data type:

In [ ]:
A_int64 = A.astype(int)  # converts to int64 by default
A_float32 = A.astype(np.float32)  # converts to float32
for array in A_int64, A_float32:
    print(array, array.dtype)

We must be careful about assigning items of different types to an array.

In [ ]:
A_int64[0, 0] = 1
print(A_int64)
A_int64[0, 0] = 0.5
print(A_int64)  # intended assignment fails
np.array([int(1), float(1)])  # will be all floating point numbers

::::{exercise}
:label: ex:ml3

Create a heterogeneous NumPy array to store both integers and strings:
```python
[0, 1, 2, 'a', 'b', 'c']
```
:::{hint}
:class: dropdown

Use the [`object` type](https://numpy.org/devdocs/reference/arrays.scalars.html#numpy.object_).
:::

::::

In [ ]:
# YOUR CODE HERE
raise NotImplementedError
heterogeneous_np_array

Be careful when creating arrays of `tuple`/`list`:

In [ ]:
%%optlite -h 350
import numpy as np
a1 = np.array([(1, 2), [3, 4, 5]], dtype=object)
print(a1.shape, a1.size)
a2 = np.array([(1, 2), [3, 4]], dtype=object)
print(a2.shape, a2.size)

NumPy provides many functions to create an array:

In [ ]:
%%optlite -h 400 -l
import numpy as np
a = [np.zeros(0)]
a.append(np.zeros(1, dtype=int))
a.append(np.zeros((2, 3, 4)))

In [ ]:
%%optlite -h 400 -l
import numpy as np
a = [np.empty(2)]
a.append(np.empty((2, 3, 4), 
                  dtype=int))

In [ ]:
%%optlite -h 400 -l
import numpy as np
a = [np.ones(2)]
a.append(np.ones((2, 3, 4),
                dtype=int))

In [ ]:
%%optlite -l -h 400
import numpy as np
a = [np.eye(2)]
a.append(np.eye(3, dtype=int))

In [ ]:
%%optlite -l -h 400
import numpy as np
a = [np.diag(range(1))]
a.append(np.diag(range(2)))
a.append(np.diag(np.ones(3), k=1))
# 1 above main diagonal

NumPy also provides functions to build an array using rules.

In [ ]:
%%optlite -l -h 400
import numpy as np
a = [np.arange(5)]
# like range but allow non-integer parameters
a.append(np.arange(2.5, 5, 0.5))

In [ ]:
%%optlite -l -h 400
import numpy as np
# can specify the number of points instead of the step size
a = [np.linspace(0, 4, 5)]
a.append(np.linspace(2.5, 4.5, 5))

In [ ]:
%%optlite -l -h 400
import numpy as np
a = np.fromfunction(lambda i, j: i * j, (3, 4))

We can also reshape an array using the `reshape` method/function:

In [ ]:
%%optlite -l -h 500
import numpy as np
a = np.arange(2 * 3 * 4)
c1 = a.reshape(2, 3, 4)
# automatically calculate the size specified as `-1`
c2 = a.reshape(2, 3, -1)

In [ ]:
%%optlite -l -h 500
import numpy as np
a = np.arange(2 * 3 * 4)
# default C ordering
c = a.reshape((2, 3, -1), order="C")
# F ordering
f = a.reshape((2, 3, -1), order="F")

The C ordering and F ordering can be implemented using list comprehension:

In [ ]:
%%optlite -l -h 500
import numpy as np
a = np.arange(2 * 3 * 4)
# last axis index changes fastest
c = np.array(
    [[[a[i*3*4+j*4+k] 
       for k in range(4)]
      for j in range(3)]
     for i in range(2)])
# first axis index changes fastest
f = np.array(
    [[[a[i+j*2+k*3*2]
       for k in range(4)]
      for j in range(3)]
     for i in range(2)])

::::{note} How to distinguish between C and F orderings easily?

When restricted to 2-dimension:

- C-style (row-major) order is like English writing, where one writes row by row.
- F-style (column-major) order is like traditional Chinese writing, where one writes column by column.

::::

`ravel` is a particular case of reshaping an array to one dimension.  
A similar function `flatten` returns a copy of the array but `ravel` and `reshape` returns a dynamic view whenever possible.

In [ ]:
array = np.arange(2 * 3 * 4).reshape(2, 3, 4)
array, array.ravel(), array.reshape(-1), array.ravel(order="F")

::::{exercise}
:label: ex:ml4

Correct the following function to print every element of an array line-by-line.
```python
def print_array_entries_line_by_line(array):
    for i in array:
        print(i)
```

::::

In [ ]:
def print_array_entries_line_by_line(array):
    # YOUR CODE HERE
    raise NotImplementedError


print_array_entries_line_by_line(np.arange(2 * 3 * 4).reshape(2, 3, 4))

### Operating on arrays

**How to verify the solution of a system of linear equations?**

Before solving the system of linear equations, let us try to verify a solution to the equations:

$$ \begin{aligned}
2x_0 + 2x_1 &= 1\\
\hphantom{2x_0 +}  2x_1 &= 1
\end{aligned}
$$

NumPy provides the function `matmul` and the operator `@` for matrix multiplication.

In [ ]:
print(np.matmul(A, np.array([0, 0])) == b)
print(A @ np.array([0, 0.5]) == b)

Note that the comparison on NumPy arrays returns a boolean array instead of a boolean value, unlike the comparison operations on lists.

To check whether all items are true, we use the `all` method.

In [ ]:
print((np.matmul(A, np.array([0, 0])) == b).all())
print((A @ np.array([0, 0.5]) == b).all())

**How to concatenate arrays?**

We will operate on an augmented matrix of the coefficients:

$$ \begin{aligned} \M{C} &= \begin{bmatrix}
\M{A} & \M{b}
\end{bmatrix}\\
&= \begin{bmatrix}
a_{00} & a_{01} & b_0 \\
a_{10} & a_{11} & b_1
\end{bmatrix} 
\end{aligned}
$$

NumPy provides functions to create block matrices:

In [ ]:
np.block?
C = np.block([A, b.reshape(-1, 1)])  # reshape to ensure same ndim
C

To stack an array along different axes:

In [ ]:
array = np.arange(1 * 2 * 3).reshape(1, 2, 3)
for concat_array in [
    array,
    np.hstack((array, array)),  # stack along the first axis
    np.vstack((array, array)),  # second axis
    np.concatenate((array, array), axis=-1),  # last axis
    np.stack((array, array), axis=0),
]:  # new axis
    print(concat_array, "\nshape:", concat_array.shape)

**How to perform arithmetic operations on a NumPy array?**

To divide all the coefficients by $2$, we can write:

In [ ]:
D = C / 2
D

Note that the above does not work for `list`.

In [ ]:
%%optlite -l -h 500
[[2, 2, 1], [0, 2, 1]] / 2

Arithmetic operations on NumPy arrays apply if the arrays have compatible dimensions. Two dimensions are compatible when
- they are equal, except for
- components equal to 1.

NumPy uses [broadcasting rules](https://numpy.org/doc/stable/user/basics.broadcasting.html#general-broadcasting-rules) to stretch the axis of size 1 up to match the corresponding axis in other arrays. `C / 2` is an example where the second operand $2$ is broadcasted to a $2$-by-$2$ matrix before the elementwise division. Another example is as follows.

In [ ]:
three_by_one = np.arange(3).reshape(3, 1)
one_by_four = np.arange(4).reshape(1, 4)
print(
    f"""
{three_by_one}
*
{one_by_four}
==
{three_by_one * one_by_four}
"""
)

In [ ]:
%%ai chatgpt -f text
Explain the broadcasting rule intuitively in one paragraph using an analogy.

Next, to subtract the second row of the coefficients from the first row:

In [ ]:
D[0, :] = D[0, :] - D[1, :]
D

Notice the use of commas to index different dimensions instead of using multiple brackets:

In [ ]:
assert (D[0][:] == D[0, :]).all()

Using this indexing technique, it is easy to extract the last column as the solution to the system of linear equations:

In [ ]:
x = D[:, -1]
x

This gives the desired solution $x_0=0$ and $x_1=0.5$ for

$$ \begin{aligned}
2x_0 + 2x_1 &= 1\\
\hphantom{2x_0 +}  2x_1 &= 1\\
\end{aligned}$$

NumPy provides many [convenient ways](https://numpy.org/doc/stable/reference/arrays.indexing.html#advanced-indexing) to index an array.

In [ ]:
B = np.arange(2 * 3).reshape(2, 3)
B, B[(0, 1), (2, 0)]  # selecting the corners using integer array

In [ ]:
B = np.arange(2 * 3 * 4).reshape(2, 3, 4)
B, B[0], B[0, (1, 2)], B[0, (1, 2), (2, 3)], B[
    :, (1, 2), (2, 3)
]  # pay attention to the last two cases

In [ ]:
assert (B[..., -1] == B[:, :, -1]).all()
B[..., -1]  # ... expands to selecting all elements of all previous dimensions

In [ ]:
B[B > 5]  # indexing using boolean array

Finally, the following function solves 2 linear equations with 2 variables.

In [ ]:
def solve_2_by_2_system(A, b):
    """Returns the unique solution of the linear system, if exists,
    else returns None."""
    C = np.hstack((A, b.reshape(-1, 1)))
    if C[0, 0] == 0:
        C = C[(1, 0), :]
    if C[0, 0] == 0:
        return None
    C[0, :] = C[0, :] / C[0, 0]
    C[1, :] = C[1, :] - C[0, :] * C[1, 0]
    if C[1, 1] == 0:
        return None
    C[1, :] = C[1, :] / C[1, 1]
    C[0, :] = C[0, :] - C[1, :] * C[0, 1]
    return C[:, -1]

In [ ]:
# tests
for A in (
    np.eye(2),
    np.ones((2, 2)),
    np.stack((np.ones(2), np.zeros(2))),
    np.stack((np.ones(2), np.zeros(2)), axis=1),
):
    print(f"A={A}\nb={b}\nx={solve_2_by_2_system(A,b)}\n")

### Universal functions

NumPy implements [universal/vectorized functions/operators](https://numpy.org/doc/stable/reference/ufuncs.html) that take arrays as arguments and perform operations with appropriate broadcasting rules. The following is an example that uses the universal function `np.sin`:

In [ ]:
@widgets.interact(a=(0, 5, 1), b=(-1, 1, 0.1))
def plot_sin(a=1, b=0):
    plt.figure(1, clear=True)
    plt.title(r"$\sin(ax+b\pi)$")
    plt.xlabel(r"$x$ (radian)")
    x = np.linspace(0, 2 * math.pi)
    plt.plot(x, np.sin(a * x + b * math.pi))  # np.sin, *, + are universal functions
    plt.show()

In addition to making the code shorter, universal functions are both efficient and flexible. Recall the Monte Carlo simulation to approximate $\pi$:

::::{exercise}
:label: ex:ml5

Explain the universal functions used in the following Monte Carlo simulation:

```python
def np_approximate_pi(n):
    in_circle = (np.random.random((n,2))**2).sum(axis=-1) < 1
    mean = 4 * in_circle.mean()
    std = 4 * in_circle.std() / n**0.5
    return np.array([mean - 2*std, mean + 2*std])
```

::::

YOUR ANSWER HERE

In [ ]:
%%ai chatgpt -f text
Explain in a paragraph why universal functions in numpy are far more efficient
than applying a function to each element of a list individually.